In [1]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score
from sklearn.metrics import classification_report, confusion_matrix, make_scorer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.linear_model import LogisticRegression

In [3]:
start_date = '2019-11-01'
end_date = '2022-10-31'

df = yf.download('TSLA', start = start_date, end = end_date, interval = '1d', progress = False, auto_adjust = True)
df.drop(columns = ['Volume'], inplace = True)
df['Target'] = np.where(df['Close'] > df['Close'].shift(1), 1, 0)
df

,Open,High,Low,Close,Target
Date,,,,,
2019-11-01,21.087999,21.098667,20.653334,20.887333,0
2019-11-04,20.986668,21.462667,20.617332,21.164667,1
2019-11-05,21.308001,21.567333,21.074667,21.148001,0
2019-11-06,21.200001,21.781334,20.966667,21.771999,1
2019-11-07,21.942667,22.766666,21.868000,22.369333,1
...,...,...,...,...,...
2022-10-24,205.820007,213.500000,198.589996,211.250000,0
2022-10-25,210.100006,224.350006,210.000000,222.419998,1
2022-10-26,219.399994,230.600006,218.199997,224.639999,1


In [4]:
df['Open_1'] = df['Open'].shift(1)
df['High_1'] = df['High'].shift(1)
df['Low_1'] = df['Low'].shift(1)
df['Close_1'] = df['Close'].shift(1)
features = ['Open_1','High_1','Low_1','Close_1']
df.dropna(inplace=True)
df[features]

,Open_1,High_1,Low_1,Close_1
Date,,,,
2019-11-04,21.087999,21.098667,20.653334,20.887333
2019-11-05,20.986668,21.462667,20.617332,21.164667
2019-11-06,21.308001,21.567333,21.074667,21.148001
2019-11-07,21.200001,21.781334,20.966667,21.771999
2019-11-08,21.942667,22.766666,21.868000,22.369333
...,...,...,...,...
2022-10-24,206.419998,214.660004,203.800003,214.440002
2022-10-25,205.820007,213.500000,198.589996,211.250000
2022-10-26,210.100006,224.350006,210.000000,222.419998


In [5]:
X = df[features].values
y = df.Target
X

array([[ 21.08799934,  21.09866714,  20.65333366,  20.88733292],
       [ 20.98666763,  21.46266747,  20.61733246,  21.16466713],
       [ 21.30800056,  21.56733322,  21.07466698,  21.14800072],
       ...,
       [210.1000061 , 224.3500061 , 210.        , 222.41999817],
       [219.3999939 , 230.6000061 , 218.19999695, 224.63999939],
       [229.77000427, 233.80999756, 222.8500061 , 225.08999634]])

In [6]:
index = int(len(y)*0.9)

Logistic Regression

In [7]:
X_train, y_train = X[:index:], y[:index:]
X_test, y_test = X[index:], y[index:]

lr= LogisticRegression()
lr.fit(X_train, y_train)
y_pred=lr.predict(X_test)

In [8]:
precision_score(y_test,y_pred)

0.5409836065573771

Naive Bayes

In [9]:
from sklearn.naive_bayes import GaussianNB
nb= GaussianNB()
nb.fit(X_train, y_train)
y_pred=nb.predict(X_test)

In [10]:
precision_score(y_test,y_pred)

0.6875

K-Nearest Neighbours

In [11]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train,y_train)
y_pred=knn.predict(X_test)

In [12]:
precision_score(y_test,y_pred)

0.5283018867924528

Decision Tree

In [13]:
from sklearn.tree import DecisionTreeClassifier
dtree= DecisionTreeClassifier(max_depth=10, random_state=101, max_features = None, min_samples_leaf = 15)
dtree.fit(X_train, y_train)
y_pred=dtree.predict(X_test)

In [14]:
precision_score(y_test,y_pred)

0.5625

Random Forest

In [15]:
from sklearn.ensemble import RandomForestClassifier
rfm = RandomForestClassifier(n_estimators=70, oob_score=True, n_jobs=-1, random_state=101, max_features = None, min_samples_leaf =30)
rfm.fit(X_train,y_train)
y_pred=rfm.predict(X_test)

In [16]:
precision_score(y_test,y_pred)

0.6428571428571429

In [17]:
df.to_csv("TESLA.csv")